In [1]:
import pandas as pd
import openpyxl
hotelsdata = pd.read_excel(r"/Users/menimalina/Desktop/uni_due/coding/python/python-project/hotels/hotels.xlsx").set_index('hotel')
hotelsdata


,Unnamed: 0,rooms,price
hotel,,,
hotel_1,0,13,273
hotel_2,1,18,92
hotel_3,2,12,141
hotel_4,3,18,157
hotel_5,4,7,298
...,...,...,...
hotel_396,395,5,212
hotel_397,396,12,68
hotel_398,397,14,102


In [2]:
print(5)

5


In [3]:
guestdata = pd.read_excel(r"/Users/menimalina/Desktop/uni_due/coding/python/python-project/hotels/guests.xlsx").set_index('guest')
guestdata

,Unnamed: 0,discount
guest,,
guest_1,0,0.09
guest_2,1,0.00
guest_3,2,0.07
guest_4,3,0.00
guest_5,4,0.10
...,...,...
guest_3996,3995,0.00
guest_3997,3996,0.15
guest_3998,3997,0.07


In [4]:
preferencesdata=pd.read_excel(r"/Users/menimalina/Desktop/uni_due/coding/python/python-project/hotels/preferences.xlsx").set_index(['guest','hotel'])
preferencesdata

Unnamed: 0  priority
guest      hotel                          
guest_1    hotel_168           0         1
           hotel_207           1         2
           hotel_222           2         3
           hotel_124           3         4
           hotel_223           4         5
...                          ...       ...
guest_4000 hotel_123       99528        28
           hotel_396       99529        29
           hotel_161       99530        30
           hotel_330       99531        31
           hotel_290       99532        32

[99533 rows x 2 columns]

In [5]:
import numpy as np

In [6]:
preferencesdata.describe()

,Unnamed: 0,priority
count,99533.000000,99533.000000
mean,49766.000000,17.011102
std,28732.846509,11.688444
min,0.000000,1.000000
25%,24883.000000,7.000000
50%,49766.000000,15.000000
75%,74649.000000,25.000000
max,99532.000000,49.000000


In [7]:
guestdata.describe()

,Unnamed: 0,discount
count,4000.000000,4000.000000
mean,1999.500000,0.093948
std,1154.844867,0.058057
min,0.000000,0.000000
25%,999.750000,0.040000
50%,1999.500000,0.100000
75%,2999.250000,0.140000
max,3999.000000,0.190000


In [8]:
hotelsdata.describe()

,Unnamed: 0,rooms,price
count,400.000000,400.000000,400.000000
mean,199.500000,11.542500,177.645000
std,115.614301,4.387633,73.159379
min,0.000000,5.000000,50.000000
25%,99.750000,8.000000,116.750000
50%,199.500000,11.000000,183.000000
75%,299.250000,15.000000,238.250000
max,399.000000,19.000000,299.000000


In [9]:
allocation = pd.DataFrame(columns=['guest_id', 'hotel_id', 'satisfaction', 'paid_price'])